In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PP.jl")
include("Estimation_PP.jl")
include("Evaluation_PP.jl")
# # include("Plot_Figures.jl")
include("Implement_All_Methods_PP.jl")
include("Models_PP.jl")

Solve_RO_one_side_exp (generic function with 1 method)

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end

## Functions

In [3]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [11]:
seed = 1;
N = 5;
N_u = 1;
K = 10;
S_train = 500;
S_test = 1000;
P_bar = 5.0;
iterations = 10;
# RO_coef_all = [0.0];
RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.15,0.2,0.25,0.3];
Random.seed!(seed)
project_dir = "Joint_Pricing_and_Promotion/"

"Joint_Pricing_and_Promotion/"

### offdiag is zero

In [12]:
offdiag_sign = "zero";
max_offdiag = 1.0;
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed);

In [13]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_dir,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);
println(this_data_file)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Joint_Pricing_and_Promotion/N=5_N_u=1_K=10_S_train=500_offdiag_sign=zero_max_offdiag=1.0/


- Generate Data

In [14]:
Input_Data = generate_Input_Data(S_train,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);

****** iter = 1 ********
****** iter = 2 ********
****** iter = 3 ********
****** iter = 4 ********
****** iter = 5 ********
****** iter = 6 ********
****** iter = 7 ********
****** iter = 8 ********
****** iter = 9 ********
****** iter = 10 ********


- Run Oracle

In [15]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

iter=1: rev_Oracle = 0.3369, price_Oracle = [0.58, 0.78, 0.6, 1.11, 0.86]
iter=2: rev_Oracle = 0.370222, price_Oracle = [0.71, 0.7, 0.71, 0.63, 0.66]
iter=3: rev_Oracle = 0.403233, price_Oracle = [0.75, 0.98, 0.6, 0.8, 0.98]
iter=4: rev_Oracle = 0.369691, price_Oracle = [0.84, 0.88, 0.97, 0.86, 0.78]
iter=5: rev_Oracle = 0.446074, price_Oracle = [0.8, 0.6, 0.77, 1.97, 1.38]
iter=6: rev_Oracle = 0.291272, price_Oracle = [1.05, 0.64, 0.52, 0.52, 1.55]
iter=7: rev_Oracle = 0.406246, price_Oracle = [0.99, 1.57, 1.63, 1.12, 0.97]
iter=8: rev_Oracle = 0.305359, price_Oracle = [0.65, 0.93, 0.84, 0.79, 0.63]
iter=9: rev_Oracle = 0.450646, price_Oracle = [0.54, 0.88, 1.31, 1.34, 0.82]
iter=10: rev_Oracle = 0.607703, price_Oracle = [0.86, 0.79, 1.0, 1.19, 1.31]


- Run ETO

In [16]:
print_flag = true
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

iter=1: rev_ETO = 0.073359, price_ETO = [4.52, 0.78, 4.56, 5.0, 0.09]
iter=2: rev_ETO = 0.350192, price_ETO = [0.71, 0.92, 0.71, 4.87, 1.26]
iter=3: rev_ETO = 0.352768, price_ETO = [0.94, 0.84, 0.97, 0.8, 0.48]
iter=4: rev_ETO = 0.1954, price_ETO = [4.34, 4.81, 0.97, 0.27, 0.97]
iter=5: rev_ETO = 0.394043, price_ETO = [0.8, 4.94, 1.24, 4.48, 1.38]
iter=6: rev_ETO = 0.263603, price_ETO = [4.97, 0.64, 0.7, 0.52, 1.55]
iter=7: rev_ETO = 0.404875, price_ETO = [0.99, 4.11, 1.63, 1.12, 0.97]
iter=8: rev_ETO = 0.239028, price_ETO = [1.12, 4.41, 0.84, 0.79, 0.74]
iter=9: rev_ETO = 0.312155, price_ETO = [0.54, 1.9, 4.9, 1.34, 3.88]
iter=10: rev_ETO = 0.517006, price_ETO = [0.08, 3.79, 1.11, 1.66, 1.31]


In [17]:
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [0.2177, 0.9459, 0.8748, 0.5286, 0.8834, 0.905, 0.9966, 0.7828, 0.6927, 0.8508]


- Run RO under different uncertainty set

In [ ]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO(RO_coef, iterations, N, N_u, K, Input_Data,0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_RO = Dict();
for RO_coef in RO_coef_all
    RST_RO_this = RST_RO["RO_coef=$(RO_coef)"]
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations];
end

println("Mean Rev_Oracle: ", mean(Rev_Oracle));
println("Mean Rev_ETO: ", mean(Rev_ETO));
Rev_RO_Avg = [mean(Rev_RO["RO_coef=$(RO_coef)"]) for RO_coef in RO_coef_all];
println("Mean Rev_RO: ", Rev_RO_Avg);

In [ ]:
ratio = Rev_RO_Avg ./ mean(Rev_ETO)
plot(RO_coef_all, ratio, marker=:o, xlabel="RO_coef", ylabel="Rev_RO_Avg / Rev_ETO_Avg", legend=false)
hline!([1.0], linestyle=:dash, color=:red, label="y=1.0")
# savefig(plt, "Rev_RO_Avg_vs_Rev_ETO_Avg.png")  # 保存为当前目录下的PNG图片

In [ ]:
plot_boxplot(RO_coef_all[1:5],Rev_ETO,Rev_RO)

### offdiag is positive

In [ ]:
# offdiag_sign = "positive";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# Random.seed!(seed)
# Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
# Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
# save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
# for RO_coef in RO_coef_all
#     Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
#     println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
# end

### offdiag is negative

In [ ]:
# offdiag_sign = "negative";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);